# Libraries

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import gc
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import StratifiedGroupKFold

# Configurations

In [2]:
class CONFIG:
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.05
    start_dt = 1100
    chg_date_3 = 1530 # 마지막 partiton

# Load training data & select

In [3]:
# Use last 2 parquets

train = pl.scan_parquet(
    f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)*2).cast(pl.Int32).alias("label"),
).filter(
    (pl.col("date_id") >= CONFIG.chg_date_3) # 이유는 모르겠지만 1100일 이후의 데이터만 가져옴
).fill_null(
    strategy="forward"
).fill_null(
    value=0 # 결측치는 일단 단순하게 0으로 처리 
)

In [4]:
train = train.collect().to_pandas()
train.head()

,id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,...,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label
0,40852762,1530,0,0,3.084694,1.153571,1.563784,0.697396,0.756759,2.580965,...,0.323897,0.601499,2.074103,0.746552,0.552013,3.071231,0.914794,0.997124,9,6
1,40852763,1530,0,1,2.232906,0.553354,1.730064,0.990195,0.611490,2.023031,...,-0.399384,-0.635306,2.092151,0.342582,0.757289,1.979042,0.967537,1.219739,9,3
2,40852764,1530,0,2,2.404948,1.532503,2.095852,0.919688,0.583715,2.330047,...,-0.006571,0.518870,-0.344441,0.641694,-0.646040,-0.506260,0.739797,-2.041514,9,-1
3,40852765,1530,0,3,1.986533,0.647099,1.687460,0.569406,1.061679,2.444131,...,-0.235901,-0.428956,-1.903627,-1.214619,-0.469500,-2.590589,-0.946317,-0.390001,9,-5
4,40852766,1530,0,4,2.742601,1.096778,1.551411,0.632113,0.368218,2.181873,...,-0.209282,-0.095182,-1.598217,0.968505,-0.705594,-1.579623,0.954296,-1.805623,9,-3


In [5]:
train = pl.DataFrame(train)

# Create Lags data from training data

In [6]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
    )
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
lags

date_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32
1531,0,-0.111643,-0.098582,-2.035532,0.772288,0.356617,-0.269562,-0.042934,0.009101,-0.106585
1531,1,1.982551,0.174802,2.422206,0.574457,0.197794,0.750299,-0.263879,-0.138759,-0.672006
1531,2,-0.658825,-0.717583,-0.137797,0.549591,0.308446,0.166129,-0.130733,-0.055674,-0.452719
1531,3,1.554449,0.562713,-0.514036,0.353216,0.235419,-0.11215,-0.087656,-0.010191,-0.223291
1531,4,0.500255,0.198204,2.345295,0.172461,0.09778,0.780662,-0.157494,-0.062953,-0.330664
…,…,…,…,…,…,…,…,…,…,…
1699,34,0.243475,0.166927,0.38494,-0.174297,-0.066046,-0.038767,-0.132337,-0.022426,-0.252461
1699,35,0.850152,0.909382,1.015314,0.235962,0.122539,0.099559,-0.249584,-0.123571,-0.46063
1699,36,0.395684,-0.292574,-3.215846,-0.535129,-0.178484,-1.80815,-0.065355,-0.000367,-0.12517


# Merge training data and lags data

In [7]:
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")
train

id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32
40852762,1530,0,0,3.084694,1.153571,1.563784,0.697396,0.756759,2.580965,0.171311,1.126353,0.536153,0.05715,11,7,76,-0.656288,2.110188,0.145784,0.0,-0.203291,0.0,-1.238222,-2.294707,-0.06356,-0.148218,1.721362,0.64558,1.477857,0.528492,1.153077,0.466157,0.145568,-0.546845,-0.694435,-0.163897,…,-0.460265,-0.415684,-0.45772,-1.333965,-2.23413,-0.352034,3.125156,0.493488,-0.9591,1.284456,-0.275493,0.0,0.0,4.188457,3.666236,0.848177,0.999516,0.417462,0.323897,0.601499,2.074103,0.746552,0.552013,3.071231,0.914794,0.997124,9,6,null,null,null,null,null,null,null,null,null
40852763,1530,0,1,2.232906,0.553354,1.730064,0.990195,0.61149,2.023031,0.319015,1.183371,0.562853,0.057789,11,7,76,-1.063518,1.037634,-0.255358,0.0,-0.318528,0.0,-1.46613,-2.160217,0.009386,0.042186,0.319811,0.14307,1.866907,1.238242,-1.986826,-0.476918,0.408439,-0.689795,-0.619278,0.081413,…,-0.24877,-0.286104,-0.455154,-1.797363,-2.535985,-0.734866,1.533782,0.033801,-0.960126,0.306505,-0.522036,0.0,0.0,1.138142,1.579439,0.179564,0.160609,-0.318671,-0.399384,-0.635306,2.092151,0.342582,0.757289,1.979042,0.967537,1.219739,9,3,null,null,null,null,null,null,null,null,null
40852764,1530,0,2,2.404948,1.532503,2.095852,0.919688,0.583715,2.330047,0.337096,1.262236,0.49605,0.073556,81,2,59,-1.001967,1.10577,-0.304426,0.0,-0.531873,0.0,-1.301579,-1.615271,0.454406,-0.188808,0.01512,-0.159487,1.379064,0.604568,0.736194,0.522007,-0.183058,-0.632819,-0.839542,-0.20955,…,-0.295958,-0.386221,-0.345102,-1.598371,-2.111468,-0.780465,0.848857,-0.152994,-1.219395,0.359229,-0.636138,0.0,0.0,0.445388,0.300118,-0.043114,-0.065761,0.200878,-0.006571,0.51887,-0.344441,0.641694,-0.64604,-0.50626,0.739797,-2.041514,9,-1,null,null,null,null,null,null,null,null,null
40852765,1530,0,3,1.986533,0.647099,1.68746,0.569406,1.061679,2.444131,0.150487,0.896543,0.705652,0.072545,4,3,11,-0.966606,0.415421,-0.528979,0.0,0.183964,0.0,-1.917427,-1.929856,-0.50175,-0.102371,-0.297677,-1.049185,0.933516,-0.60654,-0.039158,-0.27196,-0.920819,-0.926502,-0.783003,-0.13572,…,-0.396047,-0.146214,-0.228367,-2.062314,-1.421811,-0.851179,0.404792,-0.509694,-1.398495,0.231285,-0.595247,0.0,0.0,6.995454,7.133929,0.352965,0.526284,-0.349773,-0.235901,-0.428956,-1.903627,-1.214619,-0.4695,-2.590589,-0.946317,-0.390001,9,-5,null,null,null,null,null,null,null,null,null
40852766,1530,0,4,2.742601,1.096778,1.551411,0.632113,0.368218,2.181873,0.214604,1.19423,0.261056,0.045426,15,1,9,-0.895004,0.047908,-0.683484,0.0,-0.638341,0.0,-1.55811,-1.203378,-0.305831,0.251725,0.222749,-0.902276,4.226925,1.233024,-2.274672,-1.065931,-0.229211,-0.597557,-0.990242,0.173607,…,-0.445097,-0.347954,-0.470278,-1.780053,-2.290865,-0.944827,0.027896,-0.522902,-1.123492,0.120676,-0.610595,0.0,0.0,0.493589,0.560821,-1.332615,-0.965623,-0.373938,-0.209282,-0.095182,-1.598217,0.968505,-0.705594,-1.579623,0.

# Split training data and validation data

In [8]:
len_train   = train.select(pl.col("date_id")).shape[0]
valid_records = int(len_train * CONFIG.valid_ratio)
len_ofl_mdl = len_train - valid_records
last_tr_dt  = train.select(pl.col("date_id")).row(len_ofl_mdl)[0]

print(f"\n len_train = {len_train}")
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
print(f"\n---> Last offline train date = {last_tr_dt}\n")

training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 6274576

 len_ofl_mdl = 5960848

---> Last offline train date = 1690



In [9]:
validation_data

id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32
46826290,1691,0,0,3.725687,2.875351,1.902497,2.442356,2.275922,-1.900912,0.109809,-0.987361,-0.498103,-0.203913,11,7,76,-1.071316,2.596854,0.084187,-0.588666,-0.081975,-0.758741,-1.844292,-1.616732,1.129157,-0.117563,1.895694,1.055824,1.732422,1.6499,1.045879,1.16183,0.419587,-0.967041,-0.914865,-0.100731,…,-0.376398,-0.299999,-0.404779,-1.557762,-1.647366,-0.840109,1.206596,-0.180208,-0.714234,3.472888,0.481065,-0.188388,-0.256035,3.937374,6.505323,3.396123,4.425734,-0.300887,0.024018,0.26636,-1.846393,-1.319574,-1.649363,-2.37614,-2.129162,-3.700761,9,-4,-0.220147,-0.263234,0.314635,0.003016,0.004875,0.164086,0.015148,0.024748,0.016193
46826291,1691,0,1,3.344007,2.619865,0.977173,2.537985,2.014753,-2.828716,0.149373,-0.782928,-0.511855,-0.23832,11,7,76,-1.060666,0.830296,-0.40142,-0.588666,0.044371,-0.758741,-2.804641,-1.607129,1.041521,0.070675,1.561504,0.840905,1.205934,0.483422,-1.251816,-0.188141,0.736524,-0.60212,-0.716195,0.059407,…,-0.277489,-0.277824,-0.257283,-1.632294,-1.873781,-0.770637,0.438394,-0.587154,-1.011422,1.129284,-0.199034,-0.188388,-0.256035,1.37993,2.679183,0.45275,0.362266,-0.138919,-0.002569,0.175624,-2.282696,-2.430525,-3.05395,-3.764602,-2.666663,-5.0,9,-7,-1.077176,-0.22724,0.145465,0.119212,0.0406,0.258023,0.146226,0.08656,0.273702
46826292,1691,0,2,2.97724,1.891169,1.316441,2.459301,2.160374,-2.472204,0.104867,-0.831395,-0.824656,-0.38646,81,2,59,-0.593003,0.592807,-0.346058,-0.588666,-0.538496,-0.758741,-0.774373,-2.003314,-0.076213,-0.152635,0.679181,0.797249,0.531578,-0.04919,0.857706,0.68375,-0.027131,-0.433721,-0.489216,-0.192402,…,-0.272149,-0.281928,-0.299023,-1.086847,-2.924199,-0.841049,0.56977,-0.435101,-0.765014,0.443046,-0.493037,-0.188388,-0.256035,0.070608,0.048865,-0.239833,-0.179334,-0.246979,-0.112181,-0.920523,0.785028,1.981712,-2.451337,1.154909,3.141321,-0.85232,9,2,-1.273494,-0.254899,-0.731619,0.295771,0.182442,-0.137669,-0.097605,-0.00902,-0.20872
46826293,1691,0,3,1.978776,2.087344,0.962131,2.582777,2.434765,-2.637869,0.17068,-1.243193,-0.668333,-0.341923,4,3,11,-0.690289,2.167711,0.042594,-0.588666,-0.26523,-0.758741,-2.120872,-2.232578,-0.016368,-0.057444,0.375875,0.628671,0.377513,-0.282489,-0.012308,-0.427111,-0.543631,-0.631784,-0.59662,-0.043287,…,-0.051905,-0.093437,-0.072981,-2.080763,-3.644395,-0.767814,2.035217,-0.02922,-0.725214,2.076556,0.176178,-0.188388,-0.256035,1.324298,1.861361,0.140849,0.144077,0.085344,-0.032906,0.097499,-0.741609,-0.503937,0.042153,-0.994788,-0.568593,-0.002262,9,-1,-1.151294,-0.786059,0.122825,-0.228032,-0.10892,0.266399,0.264185,0.120004,0.335621
46826294,1691,0,4,2.612476,2.517823,1.177602,2.620227,2.723363,-2.819536,0.057581,-0.672236,-0.378972,-0.142323,15,1,9,-1.003598,0.337428,-0.522365,-0.588666,-0.757276,-0.758741,-1.516193,-2.868767,-0.981469,0.912641,0.902719,0.163409,4.374658,2.770107

# Save data as parquets

In [10]:
training_data.\
write_parquet(
    f"training_data.parquet"
)

In [11]:
validation_data.\
write_parquet(
    f"validation_data.parquet"
)